# Matrice de Coulomb

In [8]:
import sys
import os
sys.path.append(os.path.abspath(".."))


from utils_project import generate_csv,create_dataframe_from_xyz_files,create_X_y_from_dataframe
import pandas as pd
from ase.io import read
from dscribe.descriptors import CoulombMatrix
import numpy as np
from pathlib import Path

energy_data = pd.read_csv("../../data/energies/train.csv")
df = create_dataframe_from_xyz_files("../../data/atoms/train", csv_path=None)

# Optionnel 
xyz_dir = Path("../../data/atoms/train")
xyz_files = sorted(xyz_dir.glob("*.xyz"))
max_atoms = max(len(read(f)) for f in xyz_files)
print("Le nombre d'atome max pour une molécule est de : ", max_atoms)

cm_gen = CoulombMatrix(n_atoms_max=max_atoms, permutation="sorted_l2")

coulomb_matrices = []
energies = []

for xyz_file in xyz_files:
    atoms = read(xyz_file)
    cm = cm_gen.create(atoms)
    coulomb_matrices.append(cm.flatten())

    mol_id = int(xyz_file.stem.split('_')[-1])

    energy_row = energy_data.loc[energy_data['id'] == mol_id, 'energy']
    energies.append(energy_row.values[0])

X = np.array(coulomb_matrices)
Y = np.array(energies)


Le nombre d'atome max pour une molécule est de :  23


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(tree_method="hist", device="cpu")
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results[name] = {
        "mae": mae,
        "rmse": rmse,
        "r2": r2,
        "y_pred": y_pred
    }

    # Affichage des métriques
    print(f"\n{name}")
    print(f"  MAE  : {mae:.4f}")
    print(f"  RMSE : {rmse:.4f}")
    print(f"  R²   : {r2:.4f}")

# 🔍 Plot prédictions vs vraies valeurs
plt.figure(figsize=(14, 6))
for i, (name, res) in enumerate(results.items(), start=1):
    plt.subplot(2, 3, i)
    plt.scatter(y_test, res["y_pred"], alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red')
    plt.xlabel("Valeurs réelles")
    plt.ylabel("Prédictions")
    plt.title(name)
plt.tight_layout()
plt.show()



Linear Regression
  MAE  : 1.0795
  RMSE : 1.4305
  R²   : 0.9852

Ridge Regression
  MAE  : 1.0783
  RMSE : 1.4373
  R²   : 0.9850
